In [2]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
peft_model_id = "/home/ysx/src/AI/llm_demo/outputs/checkpoint-3000"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit=True, 
    device_map="auto",
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 8/8 [00:16<00:00,  2.10s/it]


In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    trust_remote_code=True
)

In [6]:
model = PeftModel.from_pretrained(inference_model, peft_model_id, device_map="auto")

In [15]:
peft_model_id2 = "/home/ysx/src/AI/llm_demo/outputs/checkpoint-500"

In [16]:
model = PeftModel.from_pretrained(model, peft_model_id2, device_map="auto")

In [7]:
model = model.eval()

In [17]:
response, history = model.chat(tokenizer, '''
脏腑病机是什么？回答请控制在100字以内
''', history=[])
print(response)

脏腑病机，即脏腑的病理变化，包括病位、病性、病势、病势的演变等。


In [14]:
response, history = model.chat(tokenizer, '''
请继续
''', history=history)
print(response)

清心利尿的方合方导热上通于心治上以清心导热，以清心利尿方合方导热上通于心治上以清心导热方合方导热上通于心治上以方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导热方合方导方合方，方主方方组织群方主方代导方主方病方合方病方失方合方导方病方病方代方病方代方病方导方病方病代方病方病代方病方病代方病方病方方病方主方病方方病方方方病方方病方病方病方病方病方病方病方方病方病方病方病方方病方病方病方病方病方病方病方病方病方方病方方病方方病方方病方病方病方方病方方病方方病方方方方方方方方方方方病方方方病方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方病方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方病方方方病方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方病方方方方方方病方方方方方方病方病方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方方简方方方方方方方方方方方方方气方疾病化失病制方病病化病方方病方方方制方方失病方方方症疾方方方失方症失方病方失方方方症方症失方失导方失聚方方失方方方方方方方方方方方方病方方方方问方方方症方失病病病方方病方病方病化失方病病方化方方方方方方化方方方方方病病病方方方方方方方方方方方素病方方方方方方方方方方方病病方病方化一病病部属生理代部曲部化方方部部素素化失部失失素素，素素素部生理化生理素化生理素素素部生理学素学失然素学生理代素化部失素化素病化失病化方方化方失方方方素虚方方气方化方方方失方失化病方方方方方方方方方方聚策方方病方方虚虚病，生理失代病病方方方病方方化方失化失方化失化化化化病化方病化化方方方方化方方化方方方素方虚

In [13]:
print(history)

[('\n脏腑病机是什么\n', '脏腑病机，又称脏腑病机，是脏腑发生病理改变的特定联系。脏腑的病理变化，主要表现在气和血的功能失调方面。因此，脏腑病机主要表现为生理功能失调的病理变化，而非单纯血液不足或痰浊凝聚等病理变化。\n人体是一个有机整体，人体各脏腑之间，在生理上是密切联系的，在病理上也是相互影响的。任何一个脏腑发生病变，都会影响到整个机体，而使其他脏腑发生病理改变，脏病及脏、脏病及腑、腑病及脏、腑病及腑，产生了脏腑组织之间病变的传移变化。因此，在研究脏腑病机时，不仅要注意脏腑本身的病理变化，而且要重视脏腑之间病理变化的相互影响。\n五脏病机\n五脏的阴阳、气血，是全身阴阳、气血的重要组成部分。各脏的阴阳和气血之间的关系是：气属于阳，血属于阴。气和阳，均有温煦和推动脏腑生理活动的作用，故阳与气合称为“阳气”；血和阴，均有濡养和宁静脏腑组织及精神情志的作用，故阴与血合称为“阴血”。\n但是，从阴阳、气血和各脏生理活动的关系来说，则阳和气、阴和血又不能完全等同。一般来说，脏腑的阴阳，代表着各脏生理活动的功能状态，是兴奋还是抑制，是上升或下降，还是发散或闭藏。脏腑的气血，是各脏腑生理活动的物质基础。气不仅具有推动和温煦各脏腑生理活动的作用，同时还具有重要的固摄作用。\n各脏之阴阳，皆以肾阴肾阳为根本。因此，各脏的阴阳失调，久必及肾。各脏之气血，又均化生于水谷精微，因此，各脏的气血亏虚，又与脾胃气血生化之源的关系极为密切。由于各脏的生理功能各有其特点，故各脏的阴阳失调和气血失调的病理变化也不完全相同。\n（一）心的病机\n1．心的生理病理特点：心位居上焦，开窍于舌，在体合脉，其华在面，与小肠相表里。\n心藏神，为五脏六腑之大主，又主血而外合周身之脉。心脏阴阳调和，气血充足，则心神健旺，气血环流周身，洒陈于五脏六腑，灌溉于四肢九窍，使人体各脏腑组织生生不息，藉以维持人体正常的生命活动。心包络为心之外卫，具有保护心脏，防御外邪的作用。心在脏腑中是一个重要的内脏，有“君主之官”之称。\n心的主要生理功能是主神志和主血脉。因此，心的任何病变均可出现血脉的运行异常和精神情志的改变。这些病理变化是心之阴阳气血失调的结果。所以，心之阴阳气血失调是心脏病变的内在基础。\n2．心的基本病理变化：由于阴和阳、气和血对于心主血脉和心主神志等生理功能的作用不同，故心的阴阳、气血失调因虚实寒

In [9]:
# merge base model and lora model as a standalone model.
# merged_model = model.merge_and_unload()